In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark
import findspark
findspark.init()

# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context. 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
file = "train.csv"
df_train = spark.read.csv(file,header = True,inferSchema=True)

In [3]:
df_train.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



## select 

In [4]:
df_train.select('PassengerId').show(10)

+-----------+
|PassengerId|
+-----------+
|          1|
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
|         10|
+-----------+
only showing top 10 rows



In [5]:
df_train.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [6]:
file = "train.csv"
df_train = spark.read.csv(file,header = True,inferSchema=True)
df_train = df_train.drop('PassengerId','Ticket','Name','Cabin')

In [7]:
df_train.show(4)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 4 rows



## select multiple columns

In [8]:
df_train.select('Sex','Pclass').show(3)

+------+------+
|   Sex|Pclass|
+------+------+
|  male|     3|
|female|     1|
|female|     3|
+------+------+
only showing top 3 rows



## distinct

In [9]:
df_train.select('Sex').distinct().show()

+------+
|   Sex|
+------+
|female|
|  male|
+------+



In [10]:
df_train.select('Pclass').distinct().show()

+------+
|Pclass|
+------+
|     1|
|     3|
|     2|
+------+



In [11]:
for i in df_train.columns:
    print(i , df_train.select(i).distinct().count())
    print(df_train.select(i).distinct().show())

Survived 2
+--------+
|Survived|
+--------+
|       1|
|       0|
+--------+

None
Pclass 3
+------+
|Pclass|
+------+
|     1|
|     3|
|     2|
+------+

None
Sex 2
+------+
|   Sex|
+------+
|female|
|  male|
+------+

None
Age 89
+----+
| Age|
+----+
| 8.0|
|70.0|
| 7.0|
|20.5|
|49.0|
|29.0|
|40.5|
|64.0|
|47.0|
|42.0|
|24.5|
|44.0|
|35.0|
|null|
|62.0|
|18.0|
|80.0|
|34.5|
|39.0|
| 1.0|
+----+
only showing top 20 rows

None
SibSp 7
+-----+
|SibSp|
+-----+
|    1|
|    3|
|    5|
|    4|
|    8|
|    2|
|    0|
+-----+

None
Parch 7
+-----+
|Parch|
+-----+
|    1|
|    6|
|    3|
|    5|
|    4|
|    2|
|    0|
+-----+

None
Fare 248
+-------+
|   Fare|
+-------+
|   15.5|
| 133.65|
| 8.5167|
| 29.125|
|10.4625|
| 7.0458|
|  9.475|
|11.1333|
|    0.0|
| 7.7333|
|   73.5|
|77.2875|
|   15.9|
|   11.5|
| 8.6833|
|41.5792|
|    9.5|
| 8.4042|
|14.4542|
|14.4583|
+-------+
only showing top 20 rows

None
Embarked 4
+--------+
|Embarked|
+--------+
|       Q|
|    null|
|       C|
|     

In [12]:
from pyspark.sql.functions import col, countDistinct

df_train.agg(*(countDistinct(col(c)).alias(c) for c in df_train.columns)).show()

+--------+------+---+---+-----+-----+----+--------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Embarked|
+--------+------+---+---+-----+-----+----+--------+
|       2|     3|  2| 88|    7|    7| 248|       3|
+--------+------+---+---+-----+-----+----+--------+



In [13]:
df_train.select(countDistinct(col("Sex"))).show()

+-------------------+
|count(DISTINCT Sex)|
+-------------------+
|                  2|
+-------------------+



In [14]:
df_train.select('Sex').distinct().count()

2

## value_counts

In [15]:
df_train.groupBy('Pclass').count().orderBy('Pclass').show()

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     2|  184|
|     3|  491|
+------+-----+



In [16]:
df_train.groupBy('Embarked').count().orderBy('count').show()

+--------+-----+
|Embarked|count|
+--------+-----+
|    null|    2|
|       Q|   77|
|       C|  168|
|       S|  644|
+--------+-----+



In [17]:
df_train.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [18]:
from pyspark.sql.functions import col,isnan,when,count
df_train.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_train.columns]
   ).show()

+--------+------+---+---+-----+-----+----+--------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Embarked|
+--------+------+---+---+-----+-----+----+--------+
|       0|     0|  0|177|    0|    0|   0|       2|
+--------+------+---+---+-----+-----+----+--------+



In [19]:
from pyspark.sql.functions import mean 

df_train.select(*(mean(col(c)).alias(c)  for c in df_train.columns)).show()

+------------------+-----------------+----+-----------------+------------------+-------------------+----------------+--------+
|          Survived|           Pclass| Sex|              Age|             SibSp|              Parch|            Fare|Embarked|
+------------------+-----------------+----+-----------------+------------------+-------------------+----------------+--------+
|0.3838383838383838|2.308641975308642|null|29.69911764705882|0.5230078563411896|0.38159371492704824|32.2042079685746|    null|
+------------------+-----------------+----+-----------------+------------------+-------------------+----------------+--------+



In [20]:
df_train_summ= df_train.describe()

In [21]:
df_train_summ.select('Summary','Pclass').show()

+-------+------------------+
|Summary|            Pclass|
+-------+------------------+
|  count|               891|
|   mean| 2.308641975308642|
| stddev|0.8360712409770491|
|    min|                 1|
|    max|                 3|
+-------+------------------+



In [22]:
df_train_summ.select('Summary','Age').show()

+-------+------------------+
|Summary|               Age|
+-------+------------------+
|  count|               714|
|   mean| 29.69911764705882|
| stddev|14.526497332334035|
|    min|              0.42|
|    max|              80.0|
+-------+------------------+



In [23]:
df_train.dtypes

[('Survived', 'int'),
 ('Pclass', 'int'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Fare', 'double'),
 ('Embarked', 'string')]

In [24]:
df_train.count()

891

## Null values in columns -- Handle Missing Values

In [25]:
null_cols = df_train.select([c for c in df_train.columns if df_train.filter(col(c).isNull()).count() > 0]).columns

In [26]:
null_cols

['Age', 'Embarked']

In [27]:
df_train = df_train.fillna({'Embarked':'S'})

In [28]:
null_cols = df_train.select([c for c in df_train.columns if df_train.filter(col(c).isNull()).count() > 0]).columns

In [29]:
null_cols

['Age']

## Imputer 

In [30]:
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols = null_cols,outputCols = null_cols)

In [31]:
df_new = imputer.fit(df_train).transform(df_train)


In [32]:
from pyspark.sql.functions import col,isnan,when,count
df_new.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_train.columns]
   ).show()

+--------+------+---+---+-----+-----+----+--------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Embarked|
+--------+------+---+---+-----+-----+----+--------+
|       0|     0|  0|  0|    0|    0|   0|       0|
+--------+------+---+---+-----+-----+----+--------+



In [34]:
df_new.show(4)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 4 rows



## Categorical - One hot encoding

In [38]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCols=['Sex','Embarked'], outputCols=['Num_Sex','NUM_Embarked'])
df_indexed = indexer.fit(df_new).transform(df_new)
df_indexed.show()

+--------+------+------+-----------------+-----+-----+-------+--------+-------+------------+
|Survived|Pclass|   Sex|              Age|SibSp|Parch|   Fare|Embarked|Num_Sex|NUM_Embarked|
+--------+------+------+-----------------+-----+-----+-------+--------+-------+------------+
|       0|     3|  male|             22.0|    1|    0|   7.25|       S|    0.0|         0.0|
|       1|     1|female|             38.0|    1|    0|71.2833|       C|    1.0|         1.0|
|       1|     3|female|             26.0|    0|    0|  7.925|       S|    1.0|         0.0|
|       1|     1|female|             35.0|    1|    0|   53.1|       S|    1.0|         0.0|
|       0|     3|  male|             35.0|    0|    0|   8.05|       S|    0.0|         0.0|
|       0|     3|  male|29.69911764705882|    0|    0| 8.4583|       Q|    0.0|         2.0|
|       0|     1|  male|             54.0|    0|    0|51.8625|       S|    0.0|         0.0|
|       0|     3|  male|              2.0|    3|    1| 21.075|       S

In [39]:
df_indexed = df_indexed.drop('Sex','Embarked')

In [40]:
df_indexed.show(6)

+--------+------+-----------------+-----+-----+-------+-------+------------+
|Survived|Pclass|              Age|SibSp|Parch|   Fare|Num_Sex|NUM_Embarked|
+--------+------+-----------------+-----+-----+-------+-------+------------+
|       0|     3|             22.0|    1|    0|   7.25|    0.0|         0.0|
|       1|     1|             38.0|    1|    0|71.2833|    1.0|         1.0|
|       1|     3|             26.0|    0|    0|  7.925|    1.0|         0.0|
|       1|     1|             35.0|    1|    0|   53.1|    1.0|         0.0|
|       0|     3|             35.0|    0|    0|   8.05|    0.0|         0.0|
|       0|     3|29.69911764705882|    0|    0| 8.4583|    0.0|         2.0|
+--------+------+-----------------+-----+-----+-------+-------+------------+
only showing top 6 rows



In [45]:
df_indexed.select('Survived').groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [46]:
train, test = df_indexed.randomSplit([0.8, 0.2], seed=12345)

In [48]:
train.count(),test.count()

(721, 170)

In [49]:
train.show(4)

+--------+------+----+-----+-----+------+-------+------------+
|Survived|Pclass| Age|SibSp|Parch|  Fare|Num_Sex|NUM_Embarked|
+--------+------+----+-----+-----+------+-------+------------+
|       0|     1| 2.0|    1|    2|151.55|    1.0|         0.0|
|       0|     1|18.0|    1|    0| 108.9|    0.0|         1.0|
|       0|     1|19.0|    1|    0|  53.1|    0.0|         0.0|
|       0|     1|19.0|    3|    2| 263.0|    0.0|         0.0|
+--------+------+----+-----+-----+------+-------+------------+
only showing top 4 rows



In [50]:
train.columns

['Survived',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Num_Sex',
 'NUM_Embarked']

## Use of vector assembler

In [53]:
####https://www.youtube.com/watch?v=YpI4_RrargQ
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

featureassembler=VectorAssembler(inputCols=[ 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Num_Sex',
 'NUM_Embarked'],outputCol="features")

output=featureassembler.transform(df_indexed)

In [56]:
output.select('features','Survived').show(truncate=False)

+----------------------------------------------+--------+
|features                                      |Survived|
+----------------------------------------------+--------+
|[3.0,22.0,1.0,0.0,7.25,0.0,0.0]               |0       |
|[1.0,38.0,1.0,0.0,71.2833,1.0,1.0]            |1       |
|[3.0,26.0,0.0,0.0,7.925,1.0,0.0]              |1       |
|[1.0,35.0,1.0,0.0,53.1,1.0,0.0]               |1       |
|(7,[0,1,4],[3.0,35.0,8.05])                   |0       |
|[3.0,29.69911764705882,0.0,0.0,8.4583,0.0,2.0]|0       |
|(7,[0,1,4],[1.0,54.0,51.8625])                |0       |
|[3.0,2.0,3.0,1.0,21.075,0.0,0.0]              |0       |
|[3.0,27.0,0.0,2.0,11.1333,1.0,0.0]            |1       |
|[2.0,14.0,1.0,0.0,30.0708,1.0,1.0]            |1       |
|[3.0,4.0,1.0,1.0,16.7,1.0,0.0]                |1       |
|[1.0,58.0,0.0,0.0,26.55,1.0,0.0]              |1       |
|(7,[0,1,4],[3.0,20.0,8.05])                   |0       |
|[3.0,39.0,1.0,5.0,31.275,0.0,0.0]             |0       |
|[3.0,14.0,0.0

In [58]:
all1 = output.select('features','Survived')

In [59]:
all.show(2)

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[3.0,22.0,1.0,0.0...|       0|
|[1.0,38.0,1.0,0.0...|       1|
+--------------------+--------+
only showing top 2 rows



In [60]:
train, test = all1.randomSplit([0.8, 0.2], seed=12345)

In [62]:
train.show()

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|(7,[0,1],[1.0,29....|       0|
|(7,[0,1],[1.0,29....|       0|
|(7,[0,1],[1.0,38.0])|       0|
|(7,[0,1],[1.0,39.0])|       0|
|(7,[0,1],[1.0,40.0])|       0|
|(7,[0,1],[2.0,29....|       0|
|(7,[0,1],[2.0,29....|       0|
|(7,[0,1],[2.0,29....|       0|
|(7,[0,1],[2.0,29....|       0|
|(7,[0,1],[2.0,29....|       0|
|(7,[0,1],[3.0,19.0])|       0|
|(7,[0,1],[3.0,25.0])|       1|
|(7,[0,1],[3.0,36.0])|       0|
|(7,[0,1],[3.0,49.0])|       0|
|(7,[0,1,4],[1.0,2...|       1|
|(7,[0,1,4],[1.0,2...|       1|
|(7,[0,1,4],[1.0,2...|       1|
|(7,[0,1,4],[1.0,2...|       0|
|(7,[0,1,4],[1.0,2...|       0|
|(7,[0,1,4],[1.0,2...|       0|
+--------------------+--------+
only showing top 20 rows



In [63]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features', labelCol='Survived')
lrModel = lr.fit(train)
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))


Coefficients: [-1.1400425189529793,-0.038663551985038465,-0.2511694284945116,-0.1120636915344975,0.0013273518233011317,2.795343649947557,0.3144913546197433]
Intercept: 2.020299945225205


In [64]:
pred_results=lrModel.evaluate(test)

In [70]:
pred_results.predictions.show(20)

+--------------------+--------+--------------------+--------------------+----------+
|            features|Survived|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|(7,[0,1],[2.0,29....|       0|[1.40805847173758...|[0.80345953304365...|       0.0|
|(7,[0,1,4],[1.0,2...|       1|[0.22819539808557...|[0.55680257249115...|       0.0|
|(7,[0,1,4],[1.0,2...|       1|[0.22753172217392...|[0.55663878871657...|       0.0|
|(7,[0,1,4],[1.0,2...|       0|[0.21173623547663...|[0.55273717880580...|       0.0|
|(7,[0,1,4],[1.0,3...|       1|[0.43807413214909...|[0.60780003943474...|       0.0|
|(7,[0,1,4],[1.0,4...|       0|[0.86790722221292...|[0.70431004796067...|       0.0|
|(7,[0,1,4],[1.0,5...|       1|[1.08976304633909...|[0.74833709906163...|       0.0|
|(7,[0,1,4],[2.0,1...|       0|[0.85816866939881...|[0.70227789494787...|       0.0|
|(7,[0,1,4],[2.0,3...|       0|[1.40243607852899...|[0.8025701728

In [78]:
eval_df = pred_results.predictions.select('Survived','prediction')

In [80]:
eval_df.show(2)

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       0.0|
|       1|       0.0|
+--------+----------+
only showing top 2 rows



## Accuracy of test data

In [82]:
tp  = eval_df[(eval_df.Survived == 1) & (eval_df.prediction == 1)].count()
tn  = eval_df[(eval_df.Survived == 0) & (eval_df.prediction == 0)].count()
print(tp,tn)

48 88


In [85]:
accuracy = float((tp+tn)/eval_df.count())
print(accuracy*100)

80.0


In [72]:
train.select('features').show(truncate=False)

+------------------------------------------+
|features                                  |
+------------------------------------------+
|(7,[0,1],[1.0,29.69911764705882])         |
|(7,[0,1],[1.0,29.69911764705882])         |
|(7,[0,1],[1.0,38.0])                      |
|(7,[0,1],[1.0,39.0])                      |
|(7,[0,1],[1.0,40.0])                      |
|(7,[0,1],[2.0,29.69911764705882])         |
|(7,[0,1],[2.0,29.69911764705882])         |
|(7,[0,1],[2.0,29.69911764705882])         |
|(7,[0,1],[2.0,29.69911764705882])         |
|(7,[0,1],[2.0,29.69911764705882])         |
|(7,[0,1],[3.0,19.0])                      |
|(7,[0,1],[3.0,25.0])                      |
|(7,[0,1],[3.0,36.0])                      |
|(7,[0,1],[3.0,49.0])                      |
|(7,[0,1,4],[1.0,27.0,30.5])               |
|(7,[0,1,4],[1.0,28.0,26.55])              |
|(7,[0,1,4],[1.0,28.0,35.5])               |
|(7,[0,1,4],[1.0,28.0,47.1])               |
|(7,[0,1,4],[1.0,29.0,30.0])               |
|(7,[0,1,4

In [74]:
output.select('features','Survived').show(truncate=False)

+----------------------------------------------+--------+
|features                                      |Survived|
+----------------------------------------------+--------+
|[3.0,22.0,1.0,0.0,7.25,0.0,0.0]               |0       |
|[1.0,38.0,1.0,0.0,71.2833,1.0,1.0]            |1       |
|[3.0,26.0,0.0,0.0,7.925,1.0,0.0]              |1       |
|[1.0,35.0,1.0,0.0,53.1,1.0,0.0]               |1       |
|(7,[0,1,4],[3.0,35.0,8.05])                   |0       |
|[3.0,29.69911764705882,0.0,0.0,8.4583,0.0,2.0]|0       |
|(7,[0,1,4],[1.0,54.0,51.8625])                |0       |
|[3.0,2.0,3.0,1.0,21.075,0.0,0.0]              |0       |
|[3.0,27.0,0.0,2.0,11.1333,1.0,0.0]            |1       |
|[2.0,14.0,1.0,0.0,30.0708,1.0,1.0]            |1       |
|[3.0,4.0,1.0,1.0,16.7,1.0,0.0]                |1       |
|[1.0,58.0,0.0,0.0,26.55,1.0,0.0]              |1       |
|(7,[0,1,4],[3.0,20.0,8.05])                   |0       |
|[3.0,39.0,1.0,5.0,31.275,0.0,0.0]             |0       |
|[3.0,14.0,0.0

## predictions

In [73]:
lrModel.predict(Vectors.dense([[3.0,22.0,1.0,0.0,7.25,0.0,0.0]]))

0.0

In [75]:
lrModel.predict(Vectors.dense([[[1.0,38.0,1.0,0.0,71.2833,1.0,1.0]]]))

1.0

## lrModel ## to see model specifications

In [95]:
import pyspark
print(pyspark.__version__)

3.1.2


In [99]:
spark.stop()